# IMDB Embeddings

In [13]:
from keras.datasets import imdb
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
import numpy as np

In [2]:
embedding_layer = Embedding(1000, 64)

In [3]:
max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17465344/17464789 [==============================] - 2s 0us/step


In [6]:
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [7]:
model = Sequential()
model.add(Embedding(max_features, 8, input_length=maxlen))
model.add(Flatten())  # flattens 3d to 2d
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_split=.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 84us/step - loss: 0.6759 - acc: 0.6041 - val_loss: 0.6398 - val_acc: 0.6810
Epoch 2/10
20000/20000 [==============================] - 2s 87us/step - loss: 0.5657 - acc: 0.7428 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 2s 82us/step - loss: 0.4752 - acc: 0.7808 - val_loss: 0.5113 - val_acc: 0.7384
Epoch 4/10
20000/20000 [==============================] - 2s 81us/step - loss: 0.4263 - acc: 0.8078 - val_loss: 0.5008 - val_acc: 0.7454
Epoch 5/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.3930 - acc: 0.8257 - val_loss: 0.4981 - val_acc: 0.7540
Epoch 6/10
20000/20000 [==============================] - 2s 76us/step - loss: 0.3668 - acc: 0.8394 - val_loss: 0.5013 - val_acc: 0.7532
Epoch 7/10
20000/20000 [==============================] - 1s 73us/step - loss: 0.3435 - acc: 0.8534 - val_loss: 0.5051 - val_acc:

## Raw Data

In [9]:
import os

imdb_dir = '/Users/brandon/data/aclImdb/'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

In [12]:
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [16]:
from keras.preprocessing.text import Tokenizer

# Tokenize
maxlen = 100  # max review
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [17]:
word_index = tokenizer.word_index
print('found %s unique tokens' % len(word_index))

found 88582 unique tokens


In [18]:
data = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Data tensor: ', data.shape)
print('Label tensor: ', labels.shape)

Data tensor:  (25000, 100)
Label tensor:  (25000,)


In [19]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

### GloVe Parsing

In [20]:
glove_dir = '/Users/brandon/data/glove.6B/'

embeddings_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
print('%s word vectors' % len(embeddings_index))

400000 word vectors


In [21]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector  # words not found in index are all 0s

In [22]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())  # flattens 3d to 2d
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [23]:
# loading glove embeddings
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [25]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_data=(x_val, y_val))

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 1s 5ms/step - loss: 1.5626 - acc: 0.5050 - val_loss: 0.8526 - val_acc: 0.5062
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.5788 - acc: 0.6450 - val_loss: 0.7599 - val_acc: 0.5077
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 0.3719 - acc: 0.8600 - val_loss: 2.2320 - val_acc: 0.4936
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 0.7586 - acc: 0.6500 - val_loss: 0.7455 - val_acc: 0.5058
Epoch 5/10
200/200 [==============================] - 1s 4ms/step - loss: 0.2246 - acc: 0.9550 - val_loss: 0.7652 - val_acc: 0.5006
Epoch 6/10
200/200 [==============================] - 1s 4ms/step - loss: 0.1946 - acc: 0.9500 - val_loss: 0.8573 - val_acc: 0.4945
Epoch 7/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0970 - acc: 1.0000 - val_loss: 1.2322 - val_acc: 0.4933
Epoch 8/10
200/200 [========

In [26]:
model.save_weights('pre-trained_glove_model.h5')